# Calibration Local Vol

Source : https://www.youtube.com/watch?v=tQFo8FE3jnE

On sait que ma dynamique sous black and scholes est $S_t = uS_t + \sigma S_t dW_t$
Avec un vol local ma dyamique est $dS_t = uS_t + \sigma(t,S_t) S_t dW_t$  
Comment intégrer ce modèle en MC ?

Pour bien comprends la vol local, il faut tout d'abord faire la disctinction entre ma nap de vol implicite et ma vol local. ma nap de vol implicite est observable sur le marché et est valable uniquement à cette date là. Lorsque je difuse mon spot avec MC un question ce pose: comment sélectionner la vol à chaque pas. La réponse est la vol local.

La formule de dupire afin d'obtenir la vol local est la suivante :

$\huge \sigma^2(T,K) = \frac{\frac{\partial C^u_{K,T}}{\partial T}-rC^u_{K,T}+\frac{\partial rC^u_{K,T}}{\partial K}}{\frac{1}{2} K^2 \frac{\partial C^u_{K,T}}{\partial K^2}}$

si on supose que $r = 0$ alors on obtient

$\huge \sigma^2(T,K) = \frac{\frac{\partial C^u_{K,T}}{\partial T}}{\frac{1}{2} K^2 \frac{\partial C^u_{K,T}}{\partial K^2}}$

On a des prix de call pour un nombre fini de maturité et de strike. On dit que le prix d'un call $C_i,j$ correspond à un call $C(T_i,K_j)$ *(je sais c'est à l'envers mais je prefère avoir le strike en abscise et la maturité on ordonné dans mes matrices.)*

Pour caluler $\frac{\partial C^u_{K,T}}{\partial T}$ on peut utiliser "finite derivative" ce qui reviens à faire la différence entre $C_i,j$ et $C_{i,j+1}$. J'ai donc ma dérivé par rapport aux temps

Ainsi pour $\frac{\partial C_{3,2}}{\partial T}$ nous allons faire 

$\huge \frac{\partial C_{3,2}}{\partial T} = \frac{C_{4,2} - C_{3,2}}{T_4 - T_3}$

$\begin{array}{c|ccccc}
 & K_{1} & K_{2} & K_{3} & K_{4} & K_{5} \\
\hline
T_{1} & C_{1,1} & C_{1,2} & C_{1,3} & C_{1,4} & C_{1,5} \\
T_{2} & C_{2,1} & C_{2,2} & C_{2,3} & C_{2,4} & C_{2,5} \\
T_{3} & {C_{3,1}} & \color{red}C_{3,2} & C_{3,3} & C_{3,4} & C_{3,5} \\
T_{4} & C_{4,1} & \color{orange}C_{4,2} & C_{4,3} & C_{4,4} & C_{4,5} \\
T_{5} & C_{5,1} & C_{5,2} & C_{5,3} & C_{5,4} & C_{5,5} \\
\end{array}$


Meme méthode pour $\large \frac{\partial C^u_{K,T}}{\partial K^2}$ la logique est la même que pour le gamma: on fait la différence pour $(K_i - K_{i-1}) -+ (K_i - K_{i+1})$. Je sais plus si c'est plus ou moins est flemme de réfléchir.

Ainsi pour le meme points qu'avant $C_{3,2}$ je réalise la différence:

$\begin{array}{c|ccccc}
 & K_{1} & K_{2} & K_{3} & K_{4} & K_{5} \\
\hline
T_{1} & C_{1,1} & C_{1,2} & C_{1,3} & C_{1,4} & C_{1,5} \\
T_{2} & C_{2,1} & C_{2,2} & C_{2,3} & C_{2,4} & C_{2,5} \\
T_{3} & \color{orange}{C_{3,1}} & \color{red}C_{3,2} & \color{orange}C_{3,3} & C_{3,4} & C_{3,5} \\
T_{4} & C_{4,1} & C_{4,2} & C_{4,3} & C_{4,4} & C_{4,5} \\
T_{5} & C_{5,1} & C_{5,2} & C_{5,3} & C_{5,4} & C_{5,5} \\
\end{array}$


$\huge \frac{\partial C^u_{3,2}}{\partial K^2} = \frac{2}{K_3-K_1} \cdot (\frac{C_{3,3}-C_{3,2}}{K_3-K_2}-\frac{C_{3,2}-C_{3,1}}{K_2-K_1})$

si jamais $K_3 - K_2 = K_2 - K_1$ alors $\huge = \frac{C_{3,3} - 2C_{3,2} + C_{3,1}}{\Delta K^2}$

A partir de cette matrice je peux déduire une matrice de volatilité implicite. J'obtiens donc une nap de vol. Je peux faire différent check pour la vérifié comme dit à 4:50 dans la vidéo.

J'importe mon module avec ma class VISurface qui me permet d'interpoler n'importe quel VI dans ma nap de vol.

In [8]:
from ModuleFinance import *
import numpy as np

Vol_origine = load_VI(r".\Data\Nap_Vol_SX5E.json")
r = load_rate(r".\Data\RateFr.json")
Udl = MarketData(Spot=100, Vol=Vol_origine, Taux=r, Div=0.03)
df = pd.DataFrame(Vol_origine)**2
for i, mat in enumerate(df.iterrows()):
    df.iloc[i, :] = df.iloc[i, :] * (df.index[i] / 365)
df.index = Udl.Vol.index
df.columns = Udl.Vol.columns
Udl.Vol = df
Udl._Taux

[[0.08333333333333333, 0.25, 0.5, 0.75, 1.0, 2.0, 5.0, 10.0, 30.0],
 [0.03795,
  0.03844,
  0.0385,
  0.03786,
  0.03804,
  0.034399,
  0.032262,
  0.03533,
  0.040979999999999996]]

Pour la suite il est conseillé de transformé légérement sa matrice de vol afin qu'elle soit plus simple à manipuler avec Dupire. Pour celà, on passe à fwd moneyness au lieu de spot (je suis en spot moneyness il me semble et flemme de changer) et on passe la volatilité en "total variance": ***Il multiplie la Variance par T la ou moi je ne le fait pas lors de l'interpolation linéaire (à regarder)***

$w=\sigma^2_{BS} (K,T)T$

grace à cette transformation on peut transformer la formule de Dupire comme suit:

$\huge \sigma^2 (y, T)= \frac {\frac{\partial w}{\partial T}}{1-\frac{y}{w}\frac{\partial w}{\partial y} + \frac{1}{2}\frac{\partial^2 w}{\partial y^2} + \frac{1}{4}(\frac{y^2}{w^2} - \frac{1}{w} - \frac{1}{4})(\frac{\partial w}{\partial y})^2}$

La formule à une forme similaire à $\$\Gamma$ (6min40)

## Failed Attempt to get local variance

In [3]:
#récupération de la vol local selon dupire (équation ci-dessus)
S = 100
K = 100
M = 365
r = 0.05
q = 0.02
 
M_years = M/365
F = S*np.exp((r-q)*M_years)
y = np.log(K/F)
epsilon = .01
 
w = Udl.Vol.get_vol(M,K) * M_years
partial_T = (Udl.Vol.get_vol((M+M*epsilon), K) - w) / (M*epsilon)
partial_K = (Udl.Vol.get_vol(M, (K+K*epsilon)) - w) / (K*epsilon)
partial_y = K*partial_K
second_partial_K = (Udl.Vol.get_vol(M, (K+K*epsilon)) - 2 * w + Udl.Vol.get_vol(M, (K-K*epsilon))) / (epsilon**2)
second_partial_y = K*partial_K + second_partial_K*(K**2)
 
dénominateur = 1 - (y/w)*partial_y + 1/2*second_partial_y + 1/4*((y**2)/(w**2)- 1/w - 1/4) * (partial_y**2)

rslt = np.sqrt(partial_T/dénominateur)
rslt

\\cmfrfi001\grmactions\People\Chevrier\3. Mémoire\ModuleFinance\helper.py:446: RuntimeWarning: invalid value encountered in scalar divide
  x[m] = augmented_matrice[m][n] / augmented_matrice[m][m]


0.00022264426653040704

In [4]:
Udl.Vol.get_vol(M,K)* M_years

0.022174511831521568

https://www.worldscientific.com/doi/pdf/10.1142/9789811212772_0001

$\huge \sigma^2 (T,K)= \frac {\partial _T w}{(1-\frac{y \cdot \partial _y w}{2w})^2 - \frac{(\partial _y w)^2}{4}(\frac {1}{w} + \frac{1}{4}) + \frac{\partial ^2_y w}{2}}$    avec $\partial _T w = \large \frac{\partial w}{\partial T}$ et $y = log(K/F), F= Se^{(r-q)T}$

In [5]:
S = 100
K = 100
M = 30
r = 0.05
 
T = M/365
F = S*np.exp(r*T)
y = np.log(K/F)


w = Udl.Vol.get_vol(M,K) * T
finite_derivatives_T = (Udl.Vol.get_vol(90, K)*(90/365) - w) / (60/365)
finite_derivatives_K = (Udl.Vol.get_vol(M, 120)*T - w) / (20)
second_finite_derivatives_K = (Udl.Vol.get_vol(M, 120)*T - 2 * w + Udl.Vol.get_vol(M, 80)*T) / (40**2)

#first partial deriviatives y
partial_y = T * (F*np.exp(y)) * finite_derivatives_K

#second partial derivative y
factor1 = T*F
first_part_factor2 = np.exp(y)*finite_derivatives_K
second_part_factor2 = np.exp(y)*second_finite_derivatives_K
second_partial_y = factor1 * (first_part_factor2+second_part_factor2)

#first partial derivative T
temp = T*S*np.exp(r*T + y)
term_arrobase = finite_derivatives_K*temp + finite_derivatives_T
partial_T = w + term_arrobase * T
 
dénominateur = (1 - (y*partial_y)/(2*w))**2 - ((partial_y**2)/4) * ((1/w) + (1/4)) + second_partial_y/2
rslt =partial_T/dénominateur
print(rslt)
Udl.Vol

0.0004889381951553192


,30,40,50,65,80,90,100,120,150,200
30,0.047629,0.047629,0.037558,0.021468,0.009084,0.003643,0.001049,0.002462,0.008649,0.010813
90,0.086902,0.082339,0.059676,0.033541,0.015048,0.007352,0.003594,0.003961,0.008989,0.018840
180,0.149210,0.112290,0.082181,0.048255,0.024902,0.014703,0.008741,0.006955,0.011004,0.023088
365,0.212885,0.159374,0.118458,0.074603,0.045023,0.031426,0.022175,0.015742,0.017895,0.025947
730,0.297803,0.226273,0.173193,0.117127,0.079580,0.062155,0.049587,0.037655,0.035690,0.042053
1095,0.383999,0.298562,0.233683,0.162771,0.115287,0.093656,0.077979,0.061867,0.057406,0.063267
1460,0.477091,0.375106,0.295249,0.208033,0.150978,0.125272,0.106824,0.087144,0.080784,0.088079
1825,0.579827,0.454122,0.357792,0.253895,0.187199,0.157780,0.136855,0.113456,0.104591,0.111470
2555,0.806817,0.636220,0.506991,0.368992,0.280976,0.242275,0.215012,0.182888,0.165817,0.169483
3650,1.156119,0.931309,0.761465,0.580395,0.461287,0.407613,0.368888,0.321141,0.289934,0.282805


In [6]:
#récupération de la vol local selon dupire (équation ci-dessus)
S = 100
K = 100
M = 30
r = 0.05
q = 0.02
 
M_years = M/365
F = S*np.exp((r-q)*M_years)
y = np.log(K/F)
# epsilon = .01 on drop epislon car il ne faut que utiliser des points connus.
 
w = Udl.Vol.get_vol(M,K) * M_years
partial_T = (Udl.Vol.get_vol(90, K)*(90/365) - w) / (60)
partial_K = (Udl.Vol.get_vol(M, 120)*M_years - w) / (20)
partial_y = K * partial_K
second_partial_K = (Udl.Vol.get_vol(M, 120)*M_years - 2 * w + Udl.Vol.get_vol(M, 80)*M_years) / (40**2)
second_partial_y = K*partial_K + second_partial_K*(K**2)
 
dénominateur = (1 - (y*partial_y)/(2*w))**2 - (((partial_y)**2)/4) * ((1/w) + (1/4)) + second_partial_y/2
rslt =partial_T/dénominateur
print(rslt)
Udl.Vol

1.3090101441737595e-05


,30,40,50,65,80,90,100,120,150,200
30,0.047629,0.047629,0.037558,0.021468,0.009084,0.003643,0.001049,0.002462,0.008649,0.010813
90,0.086902,0.082339,0.059676,0.033541,0.015048,0.007352,0.003594,0.003961,0.008989,0.018840
180,0.149210,0.112290,0.082181,0.048255,0.024902,0.014703,0.008741,0.006955,0.011004,0.023088
365,0.212885,0.159374,0.118458,0.074603,0.045023,0.031426,0.022175,0.015742,0.017895,0.025947
730,0.297803,0.226273,0.173193,0.117127,0.079580,0.062155,0.049587,0.037655,0.035690,0.042053
1095,0.383999,0.298562,0.233683,0.162771,0.115287,0.093656,0.077979,0.061867,0.057406,0.063267
1460,0.477091,0.375106,0.295249,0.208033,0.150978,0.125272,0.106824,0.087144,0.080784,0.088079
1825,0.579827,0.454122,0.357792,0.253895,0.187199,0.157780,0.136855,0.113456,0.104591,0.111470
2555,0.806817,0.636220,0.506991,0.368992,0.280976,0.242275,0.215012,0.182888,0.165817,0.169483
3650,1.156119,0.931309,0.761465,0.580395,0.461287,0.407613,0.368888,0.321141,0.289934,0.282805


In [7]:
print(F, w, partial_T, partial_y, second_partial_y)

100.24687958947796 8.619703098154249e-05 1.3331316938519423e-05 0.0005809884501163627 0.005435048489340356


## Correct Attempt

D'après internet:
- https://financetrainingcourse.com/education/2014/05/implied-and-local-volatility-surfaces-in-excel-final-steps/
- https://medium.com/@add.mailme/local-volatility-v-s-implied-volatility-with-practical-example-of-spx-1fc985c3ab29  
On peut aussi écrire la formule comme suit:
$$\sigma_L^2(K,T) = \frac{\sigma^2 + 2\sigma T(\frac{\partial\sigma}{\partial T} + (r-q)K\frac{\partial\sigma}{\partial K})}{(1+Kd\frac{\partial\sigma}{\partial K}\sqrt{T})^2 + \sigma K^2T((\frac{\partial^2\sigma}{\partial K^2}-d\frac{\partial\sigma}{\partial K})^2)\sqrt{T}}$$
$$Avec: d=\frac{ln(S/K)+(r+\frac{\sigma^2}{2})T}{\sigma\sqrt{T}}$$

In [8]:
Vol_origine = load_VI(r".\Data\Nap_Vol_SX5E.json")
r = load_rate(r".\Data\RateFr.json")
Udl = MarketData(Spot=100, Vol=Vol_origine, Taux=r, Div=0.03)
Udl.Vol

,30,40,50,65,80,90,100,120,150,200
30,0.761244,0.761244,0.675985,0.511070,0.332453,0.210529,0.112958,0.173090,0.324390,0.362706
90,0.593664,0.577868,0.491955,0.368819,0.247037,0.172674,0.120722,0.126750,0.190933,0.276417
180,0.550059,0.477178,0.408221,0.312810,0.224714,0.172668,0.133132,0.118757,0.149377,0.216371
365,0.461395,0.399217,0.344178,0.273135,0.212187,0.177275,0.148911,0.125466,0.133774,0.161080
730,0.385877,0.336358,0.294272,0.241999,0.199474,0.176289,0.157459,0.137213,0.133585,0.145006
1095,0.357771,0.315469,0.279095,0.232932,0.196034,0.176688,0.161224,0.143605,0.138330,0.145220
1460,0.345359,0.306229,0.271684,0.228053,0.194280,0.176969,0.163420,0.147600,0.142113,0.148391
1825,0.340537,0.301371,0.267504,0.225342,0.193494,0.177640,0.165442,0.150636,0.144631,0.149312
2555,0.339499,0.301477,0.269123,0.229593,0.200348,0.186040,0.175260,0.161638,0.153910,0.155602
3650,0.340017,0.305174,0.275947,0.240914,0.214776,0.201894,0.192065,0.179204,0.170274,0.168168


In [9]:
M = 30
K = 100
T = M/365
r = 0.05

sigma = Udl.Vol.get_vol(M,K)
d = (np.log(S/K) + ((r-q)+(sigma**2)/2)*T) / (sigma*np.sqrt(T))
partial_T = (Udl.Vol.get_vol(90,K) - sigma) / (60/365)
partial_K = (Udl.Vol.get_vol(M,120) - sigma) / (20)
second_partial_K = (Udl.Vol.get_vol(M,80) - 2*sigma + Udl.Vol.get_vol(M,120)) / (40**2)

numérateur= sigma**2 + 2*sigma*T*(partial_T + (r-q)*K*partial_K)
dénominateur1 = (1+K*d*partial_K*np.sqrt(T))**2
dénominateur2 = sigma * (K**2)*T*((second_partial_K - d*partial_K)**2)*np.sqrt(T)

variance_local = numérateur / (dénominateur1+dénominateur2)
np.sqrt(variance_local)

0.11656273886100313

In [10]:
def _get_local_vol(Udl: MarketData):
    df_local = pd.DataFrame(index=Udl.Vol.index, columns=Udl.Vol.columns)
    for M in Udl.Vol.index[1:]:
        T = M/365
        r = Udl.get_rate(T)
        # q = Udl.get_div(T)
        q = 0.02
        prev_M = Udl.Vol.index[Udl.Vol.index.get_loc(M) - 1]
        for K in Udl.Vol.columns[1:-1]:
            sigma = Udl.Vol.get_vol(M,K)
            d = (np.log(S/K) + ((r-q)+(sigma**2)/2)*T) / (sigma*np.sqrt(T))
            prev_K = Udl.Vol.columns[Udl.Vol.columns.get_loc(K) - 1]
            next_K = Udl.Vol.columns[Udl.Vol.columns.get_loc(K) + 1]

            #Différence Finit
            partial_T = (Udl.Vol.get_vol(prev_M,K) - sigma) / ((M-prev_M)/365)
            partial_K = (Udl.Vol.get_vol(M,prev_K) - sigma) / (K-prev_K)
            second_partial_K = (Udl.Vol.get_vol(M,prev_K) - 2*sigma + Udl.Vol.get_vol(M,next_K)) / ((next_K-K) * (K-prev_K))

            #Formule
            numérateur= sigma**2 + 2*sigma*T*(partial_T + (r-q)*K*partial_K)
            dénominateur1 = (1+K*d*partial_K*np.sqrt(T))**2
            dénominateur2 = sigma * (K**2)*T*((second_partial_K - d*partial_K)**2)*np.sqrt(T)
            df_local.loc[M,K] = np.sqrt(numérateur / (dénominateur1+dénominateur2))
    return df_local
vol_local = _get_local_vol(Udl)
vol_local

,30,40,50,65,80,90,100,120,150,200
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,0.729488,0.431052,0.323877,0.217281,0.155399,0.108104,0.174654,0.199651,NaN
180,NaN,0.397098,0.332639,0.251844,0.176597,0.132468,0.105607,0.153266,0.153677,NaN
365,NaN,0.318336,0.275897,0.220481,0.170139,0.138911,0.113612,0.138141,0.143982,NaN
730,NaN,0.267457,0.237826,0.200506,0.170752,0.153511,0.13656,0.130938,0.14133,NaN
1095,NaN,0.229252,0.204566,0.176742,0.160028,0.152794,0.144978,0.139096,0.132922,NaN
1460,NaN,0.210441,0.189655,0.168087,0.156882,0.153343,0.148535,0.144093,0.135819,NaN
1825,NaN,0.195765,0.179445,0.161403,0.153898,0.152096,0.148409,0.14612,0.141969,NaN
2555,NaN,0.177794,0.162807,0.145891,0.138466,0.136378,0.13226,0.12695,0.129009,NaN
3650,NaN,0.171207,0.156374,0.13904,0.130009,0.126756,0.122689,0.116713,0.115788,NaN


## Optimitation de ma nap de vol Local

La méthodo réalisé pose plusieurs problème:
- Il n'y a pas un nap de VI unique mais une quantité infini.
- Comme je calcul la dérivite via "finite derivative" le moindre changement de prix change drastiquement ma dérivé
- j'ai un nombre de prix observable unique
- quels prix prendre celui de la cloture la moyenne des prix du jours ? le prix n'est jamais le meme et peut avoir des erreurs dedans 

Le prix du marché est noté $C^{Obs}(K_i,T_j)$  
Le prix en vol local est noté  $C^{LV}(K_i,T_j,\sigma(t,S))$  

Ainsi on peut parfois laissé un mismatch afin d'avoir un modèle plus proche de la réalité. Ainsi:  
$C^{Obs}(K_i,T_j) = C^{LV}(K_i,T_j,\sigma(t,S)) + \epsilon_{i,j}$ ou  
$C^{Obs}_{i,j} = C^{\sigma(t,s)}_{i,j}+ \epsilon_{i,j}$  
Cet écart peut venir du bid-ask, la source des datas, l'heure des datas...


In [11]:
def transforme_vol_to_price(volatility_local: VISurface, volatility_bs: VISurface):
    matrice_rlst = pd.DataFrame(index=volatility_local.index, columns=volatility_local.columns)
    for M in volatility_local.index:
        T = M/365
        for K in volatility_local.columns:
            Call = CallOption(Udl,K,T)
            vol_local = volatility_local.loc[M,K]
            vol_bs = volatility_bs.loc[M,K]

            Udl.Vol = vol_local
            prix_local = Call.price()
            
            Udl.Vol = vol_bs
            prix_bs = Call.price()

            matrice_rlst.loc[M,K] = prix_local - prix_bs
    return matrice_rlst

In [12]:
#epsilon Prix
Udl = MarketData(100,Vol_origine,load_rate(r".\Data\RateFr.json"),0.02)
C_espsilon = transforme_vol_to_price(vol_local, Udl.Vol)
C_espsilon

,30,40,50,65,80,90,100,120,150,200
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,0.036226,-0.008508,-0.029258,-0.074437,-0.129371,-0.24725,0.065245,0.000042,NaN
180,NaN,-0.015858,-0.040416,-0.117681,-0.305211,-0.592268,-0.755238,0.200957,0.00014,NaN
365,NaN,-0.069823,-0.137597,-0.316162,-0.65756,-1.04031,-1.354585,0.243868,0.009706,NaN
730,NaN,-0.198599,-0.331255,-0.594314,-0.849676,-0.96702,-1.098123,-0.264831,0.078266,NaN
1095,NaN,-0.436006,-0.702965,-1.15901,-1.395119,-1.24928,-1.012874,-0.26528,-0.126941,NaN
1460,NaN,-0.725108,-1.082333,-1.590297,-1.722291,-1.408932,-1.037998,-0.2475,-0.246393,NaN
1825,NaN,-1.049764,-1.464493,-2.004962,-2.046297,-1.666353,-1.285589,-0.358477,-0.143777,NaN
2555,NaN,-1.796596,-2.385226,-3.21178,-3.646806,-3.601798,-3.585876,-3.179078,-1.853389,NaN
3650,NaN,-2.723611,-3.499672,-4.648577,-5.519652,-5.862711,-6.182048,-6.376566,-5.258779,NaN


In [13]:
import plotly.graph_objects as go
go.Figure(data=[go.Surface(z=C_espsilon.values, y=C_espsilon.index, x = C_espsilon.columns)]).show()

On foit pas que pour la vol longterm c'est ignoble alors que pour la vol courte de maturité la vol est OK

In [14]:
#epsilon Vol
df_diff = vol_local - pd.DataFrame(Vol_origine)
df_diff

,30,40,50,65,80,90,100,120,150,200
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,0.151621,-0.060903,-0.044943,-0.029757,-0.017275,-0.012618,0.047905,0.008717,NaN
180,NaN,-0.08008,-0.075582,-0.060966,-0.048117,-0.0402,-0.027525,0.034508,0.0043,NaN
365,NaN,-0.080881,-0.068281,-0.052654,-0.042048,-0.038363,-0.035299,0.012675,0.010208,NaN
730,NaN,-0.068901,-0.056447,-0.041493,-0.028722,-0.022777,-0.020899,-0.006275,0.007745,NaN
1095,NaN,-0.086217,-0.07453,-0.05619,-0.036005,-0.023894,-0.016246,-0.004509,-0.005408,NaN
1460,NaN,-0.095789,-0.082028,-0.059967,-0.037397,-0.023626,-0.014884,-0.003507,-0.006293,NaN
1825,NaN,-0.105606,-0.088059,-0.063939,-0.039596,-0.025544,-0.017032,-0.004515,-0.002662,NaN
2555,NaN,-0.123683,-0.106317,-0.083703,-0.061882,-0.049662,-0.043,-0.034688,-0.024901,NaN
3650,NaN,-0.133966,-0.119573,-0.101874,-0.084767,-0.075138,-0.069375,-0.062491,-0.054486,NaN


In [15]:
go.Figure(data=[go.Surface(z=vol_local.values, y=vol_local.index, x = vol_local.columns)]).show()

In [16]:
fig = go.Figure(data=[go.Surface(z=Udl.Vol.values,y=vol_local.index, x = vol_local.columns)]).show()

AttributeError: 'numpy.float64' object has no attribute 'values'

In [ ]:
fig = go.Figure(data=[go.Surface(z=df_diff.values,y=df_diff.index, x = df_diff.columns)]).show()

**Ce qu'on cherche à optimiser ici est la nap de BS et non la nap de vol local. Il existe Thin Plate Splines qui permet en partie de résoudre ce pb.**

Par exemple on à fait des cubic splines sur le strike ainsi il passe par tous les points connus. ce qu'on peut faire appliqué à mes splines la meme logique que la méthode des moindres carré.  
Ainsi pour un smile donnée (une tranche de maturité) j'ai : $\min \limits_{\sigma(T,S) \in S}\sum\limits_{i=1}^{N}(C^{Obs}_{i,j}-C^{\sigma(t,s)}_{i,j})^2$

on a pas de solution unique mais alors comment faire ?
Pour cela on va imposer une pénalité:  
$\min \limits_{\sigma(T,S) \in S}\sum\limits_{i=1}^{N}(C^{Obs}_{i,j}-C^{\sigma(t,s)}_{i,j})^2 + \lambda \int_{0}^{\infty} (\frac{\partial^2\sigma(T,K)}{\partial K^2})dK$

Ainsi cette équation se split en 2:
- la première partie réprésente l'écart avec les prix observés
- la 2nd partie correspond quel point ma nap est smooth  

Lambda représente a quel point je sacrifie ma fidélèté au data pour le smoothness de ma nap de vol

Ainsi je smooth mon cubic spline, j'ai fait du natural cubic spline car la dérivés de mes end points sont 0

Le processus suivit est actuellement pour un smile je dois donc le réalisé pour l'ensemble des smiles  
$\min \limits_{\sigma(T,S) \in S}\color{red}\sum\limits_{i=1}^{N}\color{default}\sum\limits_{i=1}^{M}(C^{Obs}_{i,j}-C^{\sigma(t,s)}_{i,j})^2$

Actuellement tous les écarts entre prix calculé et observé et pondéré de la même manière mais si je veux fitter à un endroit ou alors je pense que certaine donnée sont fausse je peux introduire de la pondération. Selon moi, on préfère bien fitté ATM, 30j plus tot que 30%,10Y.  
$\min \limits_{\sigma(T,S) \in S}\sum\limits_{i=1}^{N}\sum\limits_{i=1}^{M}\color{red}w_{i,j}\color{default}(C^{Obs}_{i,j}-C^{\sigma(t,s)}_{i,j})^2$  

Je réintegre la pénalité mais cette fois en 3d  
$\min \limits_{\sigma(T,S) \in S}\sum\limits_{i=1}^{N}\sum\limits_{i=1}^{M}w_{i,j}(C^{Obs}_{i,j}-C^{\sigma(t,s)}_{i,j})^2 + \lambda \lVert L\sigma \rVert^2_2$ avec $L = \frac{\partial^2}{\partial K^2}$ et $L = \frac{\partial^2}{\partial K^2} + 2\frac{\partial^2}{\partial K\partial T} + \frac{\partial^2}{\partial T^2}$ si on est en 2D.

15min30 dans la vidéo mais j'ai pas tout compris askip dans notre car le meilleur L est $L =\frac{\partial^2}{\partial K^2} + \frac{\partial}{\partial T}$

## Variation Calculus

C'est comme le normal Calculus sauf que ma fonction n'est pas x,y mais une fonction de fonction. Ainsi on recherche une fonction qui minimise mes fonctions. exemple : Euler Lagrange Equation et Integral of the First Kind :  
$g(t) = \int_{a}^{b} K(t,s)f(s)ds$ résoudre cette intégrale revient a faire $g = K \cdot f$ qui est un système d'équation linéaire

least square qu'on connais est :  
$\min \limits_{b} \sum \limits_{i=1}^N (y_i - bx_i)^2$  
en Variation Calculs:  
$\min \limits_{\beta \in R^n} \lVert y - X\beta \rVert^2_2$ avec $\beta = (X'X)^{-1}X'y$  

je rajoute ma contrainte:  
$\min \limits_{\beta \in R^n} \lVert y - X\beta \rVert^2_2 + \lambda \lVert L\beta \rVert^2_2$ ainsi $\beta = (X'X + \lambda L'L)^{-1}X'y$  

J'ai peux résoudre simplement et betement en calculant l'inverse et multipliant mes matrices mais il existe des méthodes plus efficace comme par décompostion tel que SVD (single value deconposition) mais ici faut que j'utilise le généralise SVD

Il faut aussi fitté le $L$. Si on a chosit $L = \frac{\partial^2}{\partial K^2}$ alors on doit résoudre:  
\begin{align*}
\begin{bmatrix}
\color{red}1 & \color{red}-2 & \color{red}1 & 0 & \cdots & 0 \\
0 & 1 & -2 & 1 & \cdots & 0 \\
0 & 0 & 1 & -2 & \cdots & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 & \cdots & 1 \\
\end{bmatrix}
&
\begin{bmatrix}
\color{red}\sigma_0 \\
\color{red}\sigma_1 \\
\color{red}\sigma_2 \\
\vdots \\
\sigma_{n-1} \\
\sigma_n
\end{bmatrix}
&= 
\begin{bmatrix}
\color{red}\sigma_0 - 2\sigma_1 + \sigma_2 \\
\sigma_1 - 2\sigma_2 + \sigma_3 \\
\vdots \\
\sigma_{n-2} - 2\sigma_{n-1} + \sigma_{n}
\end{bmatrix}
\end{align*}  

il faut ensuite diviser le résultat par le changement de $K$ au carré. La deuxieme matrice fait $(N+1)$ (de 0 à n). La matrice résult fait $(n-1)$. Ainsi ma première matrice fait $(n-1)*(n+1)$. Dans le beta comme je fait la matrice multiplé par sa transposé ça devriendra une matrice carrré de $(n+1)$. Si comme pour les cubics spline je veux spécifier mes dériviés secondes à mes ends points je peux rajouter 2 lignes à ma première matrice.  

Si je prends $L = \frac{\partial}{\partial K}$:  
\begin{align*}
\begin{bmatrix}
-1 & 1 & 0 & \cdots & 0 \\
0 & -1 & 1 & \cdots & 0 \\
0 & 0 & -1 &  \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & 1 \\
\end{bmatrix}
&
\begin{bmatrix}
\sigma_0 \\
\sigma_1 \\
\sigma_2 \\
\vdots \\
\sigma_{n-1} \\
\sigma_n
\end{bmatrix}
&= 
\begin{bmatrix}
\sigma_1 - \sigma_0 \\
\sigma_2 - \sigma_1 \\
\vdots \\
\sigma_{n} - \sigma_{n-1} 
\end{bmatrix}
\end{align*}  

25min
Jusqu'a maintenant on supposé qu'on connaissait $\lambda$ mais on ne le connait pas. il esxite plusieurs moyen de le définir : 
- Discrepancy Principle
- L-Curve criteria
- Genrealised Cross Validation (pas sur qu'il soit bon car enlève les valeurs extremes)